In [ ]:
from pyspark.sql.functions import first, col, max as spark_max, lit, trim, current_timestamp, regexp_replace, when, row_number, coalesce, to_timestamp, lower, size, split, element_at
from pyspark.sql import DataFrame, Window
from pyspark.sql.types import DoubleType, TimestampType, IntegerType
from delta.tables import DeltaTable
from pyspark.sql import functions as F, types as T , Window
from collections import Counter, defaultdict
import unicodedata
import re, unicodedata
import time
from pyspark import StorageLevel
import os

In [ ]:
# Spark otimizado para DataFrames até 300 linhas

spark.conf.set("spark.sql.shuffle.partitions", 1)   # Menos shuffle
spark.conf.set("spark.sql.execution.arrow.pyspark.enabled", "true")  # Mais rápido Pandas ⇆ Spark
spark.conf.set("spark.databricks.delta.merge.enableLowShuffle", "true")  # Merge rápido em tabelas pequenas

# Compactação automática desligada (não faz sentido para tabelas pequenas)
spark.conf.set("spark.databricks.delta.optimizeWrite.enabled", "false")
spark.conf.set("spark.databricks.delta.autoCompact.enabled", "false")

spark.conf.set("spark.sql.shuffle.partitions", 1)

spark.conf.set("spark.sql.autoBroadcastJoinThreshold", -1)


In [ ]:
spark.conf.set("spark.sql.session.timeZone", "UTC")
spark.conf.get("spark.sql.session.timeZone")

In [ ]:
# ─── 2) Caminhos dos Lakehouses ──────────────────────────────────────────────
# Lakehouse Bronze (onde estão as tabelas originais)
bronze_lakehouse_path = (
    "abfss://<GUID>@"
    "onelake.dfs.fabric.microsoft.com/"
    "<GUID>/Tables"
)

# Lakehouse Prata (onde você está rodando este notebook)
prata_lakehouse_path = (
    "abfss://<CONTAINER>@onelake.dfs.fabric.microsoft.com/<WORKSPACE_ID>/<PATH>"
)

print("📂 Bronze:", bronze_lakehouse_path)
print("📂 Prata :", prata_lakehouse_path)



In [ ]:
# ===========================================================
# 🧹 EXCLUSÃO DE REGISTROS INSERIDOS NESTE MÊS (FUSO BRT)
# ===========================================================

from delta.tables import DeltaTable
from pyspark.sql import functions as F
from datetime import datetime
from zoneinfo import ZoneInfo

# Timezone de Brasília
BRT = ZoneInfo("America/Sao_Paulo")

# Lista de tabelas Bronze a limpar
tabelas_para_limpar = [
    "venda_prata",
    "unidade_prata",
    "reservas_workflow_prata",
    "reservas_registros_flags_prata",
    "reserva_prata",
    "repasse_prata",
    "precadastros_workflow_prata",
    "precadastro_prata",
    "leads_workflow_prata",
    "lead_prata",
    "imobiliarias_prata",
    "corretores_prata",
    "campos_adicionais_prata"
]

# Primeiro dia do mês atual (ajustado para Brasília)
data_corte = datetime(2025, 10, 31, 0, 0, 0, tzinfo=BRT)
print(f"🕒 Considerando registros com ingest_ts >= {data_corte} (fuso BRT)\n")

for tabela in tabelas_para_limpar:
    print(f"🧹 Limpando registros recentes da tabela: {tabela}")

    try:
        if not DeltaTable.isDeltaTable(spark, f"Tables/{tabela}"):
            print(f"⚠️  Tabela {tabela} não é Delta ou não existe — pulando.\n")
            continue

        delta_tbl = DeltaTable.forName(spark, tabela)
        # Condição: registros inseridos a partir do primeiro dia do mês
        condicao = F.col("ingest_ts") >= F.lit(data_corte)
        # Executa o delete
        delta_tbl.delete(condicao)
        print(f"✅ Registros deste mês removidos com sucesso de {tabela}.\n")

    except Exception as e:
        print(f"❌ Erro ao limpar {tabela}: {e}\n")

print("──────────────────────────────────────────────")
print("🧩 Limpeza concluída em todas as tabelas listadas.")


In [ ]:
# ===========================================================
# 🧹 DROP DAS TABELAS PRATA (APENAS NO PRIMEIRO DIA DO MÊS)
# ===========================================================

from datetime import datetime, timedelta
from zoneinfo import ZoneInfo

# Timezone BRT
BRT = ZoneInfo("America/Sao_Paulo")
agora = datetime.now(BRT)

# Identificar primeiro dia do mês
primeiro_dia_mes = datetime(agora.year, agora.month, 1, tzinfo=BRT)

# ===========================================================
# 🔍 VERIFICAÇÃO: só roda o DROP se for o primeiro dia
# ===========================================================

if agora.date() == primeiro_dia_mes.date():

    print("✅ Hoje é o PRIMEIRO dia do mês. Executando DROP das tabelas PRATA...\n")

    tabelas_para_drop = [
        "venda_prata",
        "unidade_prata",
        "reservas_workflow_prata",
        "reservas_registros_flags_prata",
        "reserva_prata",
        "repasse_prata",
        "precadastros_workflow_prata",
        "precadastro_prata",
        "leads_workflow_prata",
        "lead_prata",
        "imobiliarias_prata",
        "corretores_prata",
        "campos_adicionais_prata"
    ]

    # Executa o DROP TABLE para cada tabela
    for tabela in tabelas_para_drop:
        print(f"🧹 Dropando tabela: {tabela}")
        try:
            spark.sql(f"DROP TABLE IF EXISTS {tabela}")
            print(f"🗑️ Tabela {tabela} removida com sucesso.\n")

        except Exception as e:
            print(f"❌ Erro ao dropar a tabela {tabela}: {e}\n")

    print("──────────────────────────────────────────────")
    print("🧩 DROP TOTAL (PRATA) finalizado com sucesso.")

else:
    print("🔕 Não é o PRIMEIRO dia do mês. Pulando o DROP das tabelas PRATA.\n")


In [ ]:
from delta.tables import DeltaTable
from pyspark.sql import functions as F

def ler_incremental_bronze(
    nome_tabela: str,
    path_bronze: str,
    path_prata: str,
    id_col: str,
    ids_alternativos: list = None
):
    """
    Lê o incremental da Bronze SEM CDF.
    Sempre usa referencia_data da Bronze como base de incremental.
    Como as tabelas PRATA são dropadas no 1º dia do mês, PRATA inexistente = full load.
    """

    print(f"\n🔍 Lendo incremental da tabela '{nome_tabela}'...")

    if ids_alternativos is None:
        ids_alternativos = [id_col]

    # 1️⃣ Se a PRATA não existe — FULL LOAD
    if not DeltaTable.isDeltaTable(spark, path_prata):
        print(f"⚠️ PRATA '{nome_tabela}' inexistente — lendo Bronze completa (primeira carga).")
        df_bronze_full = spark.read.format("delta").load(path_bronze)
        print(f"✅ {nome_tabela}: Bronze carregada completamente — {df_bronze_full.count()} registros.")
        return df_bronze_full

    # 2️⃣ PRATA existe → fazer incremental baseado no MAX(referencia_data)
    df_bronze_base = spark.read.format("delta").load(path_bronze)
    df_prata_base  = spark.read.format("delta").load(path_prata)

    # Verifica se existe referencia_data nas duas
    bronze_cols = [c.lower() for c in df_bronze_base.columns]
    prata_cols  = [c.lower() for c in df_prata_base.columns]

    if "referencia_data" not in bronze_cols:
        print("⚠️ Bronze não possui referencia_data → FULL REFRESH da Bronze.")
        return df_bronze_base

    if "referencia_data" not in prata_cols:
        print("⚠️ Prata não possui referencia_data → FULL REFRESH da Bronze.")
        return df_bronze_base

    # 3️⃣ Descobre a maior referencia_data da PRATA
    ultima_ref = (
        df_prata_base
        .agg(F.max("referencia_data").alias("max_ref"))
        .collect()[0]["max_ref"]
    )

    if not ultima_ref:
        print("⚠️ PRATA sem referencia_data válida → FULL REFRESH.")
        return df_bronze_base

    print(f"📅 Última referencia_data processada na PRATA: {ultima_ref}")

    # 4️⃣ Filtrar Bronze NOVA (incremental)
    df_incremental_bronze = df_bronze_base.filter(
        F.col("referencia_data") > F.lit(ultima_ref)
    )

    print(f"📈 Registros incrementais encontrados: {df_incremental_bronze.count()}")

    # 🔄 Anti-join para evitar duplicados id + referencia_data
    df_prata_ref = df_prata_base.select(id_col, "referencia_data")

    df_incremental_final = (
        df_incremental_bronze.alias("b")
        .join(
            df_prata_ref.alias("p"),
            (F.col(f"b.{id_col}") == F.col(f"p.{id_col}")) &
            (F.col("b.referencia_data") == F.col("p.referencia_data")),
            "leftanti"
        )
    )

    print(f"✅ Incremental pronto — {df_incremental_final.count()} registros únicos.")
    return df_incremental_final


## [**Fatos**]

In [ ]:
# ================================================================
# ⚙️ CAMINHOS FÍSICOS DAS TABELAS BRONZE
# ================================================================
path_leads         = f"{bronze_lakehouse_path}/leads_bronze_cvdw"
path_repasse       = f"{bronze_lakehouse_path}/repasses_bronze_cvdw"
path_reserva       = f"{bronze_lakehouse_path}/reservas_bronze_cvdw"
path_precadastro   = f"{bronze_lakehouse_path}/precadastros_bronze_cvdw"

# ================================================================
# ⚙️ CAMINHOS FÍSICOS DAS TABELAS PRATA
# ================================================================
path_leads_prata        = f"{prata_lakehouse_path}/lead_prata"
path_repasse_prata      = f"{prata_lakehouse_path}/repasse_prata"
path_reserva_prata      = f"{prata_lakehouse_path}/reserva_prata"
path_precadastro_prata  = f"{prata_lakehouse_path}/precadastro_prata"


In [ ]:
# ------------------- LEADS -------------------
df_leads_bronze = ler_incremental_bronze(
    nome_tabela="leads",
    path_bronze=path_leads,
    path_prata=path_leads_prata,
    id_col="idlead"
)
if df_leads_bronze is not None and not df_leads_bronze.rdd.isEmpty():
    df_leads_bronze = df_leads_bronze.repartition(2, F.col("idlead"))  # 👈 define 4 partições fixas
    print(f"✅ df_leads_bronze criado com sucesso — {df_leads_bronze.count()} registros incrementais.\n")
else:
    print("⚠️ Nenhum dado incremental carregado para 'leads'.\n")

In [ ]:
# ------------------- PRECADASTRO -------------------
df_precadastro_bronze = ler_incremental_bronze(
    nome_tabela="precadastro",
    path_bronze=path_precadastro,
    path_prata=path_precadastro_prata,
    id_col="idprecadastro"
)
if df_precadastro_bronze is not None and not df_precadastro_bronze.rdd.isEmpty():
    print(f"✅ df_precadastro_bronze criado com sucesso — {df_precadastro_bronze.count()} registros incrementais.\n")
else:
    print("⚠️ Nenhum dado incremental carregado para 'precadastro'.\n")


In [ ]:
# ------------------- RESERVA -------------------
df_reserva_bronze = ler_incremental_bronze(
    nome_tabela="reserva",
    path_bronze=path_reserva,
    path_prata=path_reserva_prata,
    id_col="idreserva"
)
if df_reserva_bronze is not None and not df_reserva_bronze.rdd.isEmpty():
    print(f"✅ df_reserva_bronze criado com sucesso — {df_reserva_bronze.count()} registros incrementais.\n")
else:
    print("⚠️ Nenhum dado incremental carregado para 'reserva'.\n")

In [ ]:
# ------------------- REPASSE -------------------
df_repasse_bronze = ler_incremental_bronze(
    nome_tabela="repasse",
    path_bronze=path_repasse,
    path_prata=path_repasse_prata,
    id_col="idrepasse"
)
if df_repasse_bronze is not None and not df_repasse_bronze.rdd.isEmpty():
    print(f"✅ df_repasse_bronze criado com sucesso — {df_repasse_bronze.count()} registros incrementais.\n")
else:
    print("⚠️ Nenhum dado incremental carregado para 'repasse'.\n")

In [ ]:
def normalizar_nomes_colunas(df: DataFrame) -> DataFrame:
    """
    Normaliza os nomes das colunas:# Registrar as views no catálogo temporário
df_leads_bronze.createOrReplaceTempView("leads_bronze")
df_repasse_bronze.createOrReplaceTempView("repasse_bronze")
df_reserva_bronze.createOrReplaceTempView("reserva_bronze")
df_precadastro_bronze.createOrReplaceTempView("precadastro_bronze")
      - Remove acentos e pontuações
      - Substitui espaços e hífens por "_"
      - Remove ., ,, ?, !, :, ;, etc.
      - Converte tudo para minúsculo
    Retorna um novo DataFrame com colunas renomeadas
    """

    def limpar_nome(col_name: str) -> str:
        # remove acentos
        sem_acentos = unicodedata.normalize("NFKD", col_name).encode("ASCII", "ignore").decode("utf-8")
        # remove pontuações e substitui por nada
        sem_pontuacao = re.sub(r"[.,;:!?\"'()%$#@/\\-]", "", sem_acentos)
        # troca espaços e múltiplos underscores por _
        com_underscore = re.sub(r"\s+", "_", sem_pontuacao)
        # remove underscores duplos ou no início/fim
        limpo = re.sub(r"_+", "_", com_underscore).strip("_")
        # tudo minúsculo
        return limpo.lower()

    # Aplica renomeação de forma iterativa
    for old_name in df.columns:
        new_name = limpar_nome(old_name)
        if new_name != old_name:
            df = df.withColumnRenamed(old_name, new_name)

    print(f"✅ Colunas normalizadas ({len(df.columns)} colunas)")
    return df


In [ ]:
df_precadastro_bronze = normalizar_nomes_colunas(df_precadastro_bronze)
df_reserva_bronze     = normalizar_nomes_colunas(df_reserva_bronze)
df_repasse_bronze     = normalizar_nomes_colunas(df_repasse_bronze)
df_leads_bronze       = normalizar_nomes_colunas(df_leads_bronze)

#### [**Tratar Valores e Tipagem**]

In [ ]:
# 🔹 Pré-compilação de regex — evita recriar padrões a cada chamada
_RE_NON_ALNUM = re.compile(r"[^A-Za-z0-9_]+")
_RE_MULTI_UNDERSCORE = re.compile(r"_+")

def _norm_basic(s: str) -> str:
    """
    Normaliza nomes de colunas para comparação robusta (sem acentos, espaços, símbolos).
    Exemplo: "Data de Nascimento" -> "data_de_nascimento"
    """
    if not isinstance(s, str):
        return ""
    s2 = unicodedata.normalize("NFKD", s).encode("ascii", "ignore").decode("ascii")
    s2 = _RE_NON_ALNUM.sub("_", s2.strip().lower())
    s2 = _RE_MULTI_UNDERSCORE.sub("_", s2).strip("_")
    return s2


def _map_list_by_normalization(df_cols, lista):
    """
    Mapeia nomes normalizados de 'lista' para nomes reais do DataFrame.
    Exemplo: ['data_cadastro'] pode casar com ['Data Cadastro', 'data_cad'] etc.
    """
    if not lista:
        return []

    # 🔹 Monta dicionário normalizado → nomes reais
    norm2real = {}
    for c in df_cols:
        norm = _norm_basic(c)
        norm2real.setdefault(norm, []).append(c)

    resolved = []
    seen = set()

    for item in lista:
        key = _norm_basic(item)
        hits = norm2real.get(key)
        if not hits:
            continue
        for h in hits:
            if h not in seen:
                resolved.append(h)
                seen.add(h)

    return resolved


In [ ]:
# =====================================================================
# 6B) TRATAR_VALORES_E_DATAS (versão avançada Hudson)
# =====================================================================
from pyspark.sql import functions as F, types as T

def tratar_valores_e_datas(
    df,
    colunas_valor: list = None,
    colunas_data: list = None,
    excluir_valor: list = None,
    excluir_data: list = None,
    forcar_valor: list = None,
    forcar_data: list = None,
    zero_um_como_nulo: bool = False,
    explicit_prefix: str = None,
    limitar_ao_prefixo: bool = None,
    auto_prefix_listas: bool = None,
    match_normalizado_nas_listas: bool = True,
):
    """
    Converte colunas de valor -> Double e colunas de data -> Timestamp,
    limpando apenas as células inválidas de cada coluna (nunca a linha toda).
    Corrige '... .0' em datas e filtra anos fora de [1900,2100].
    """
    df_tratado = df
    dtypes = dict(df.dtypes)
    df_cols = df.columns

    # Defaults
    if limitar_ao_prefixo is None:
        limitar_ao_prefixo = bool(explicit_prefix)
    if auto_prefix_listas is None:
        auto_prefix_listas = bool(explicit_prefix)
    prefix = explicit_prefix or ""

    # Helpers
    def base_name(c):
        return c[len(prefix):] if (prefix and c.startswith(prefix)) else c

    # Inicializa listas
    colunas_valor = list(colunas_valor or [])
    colunas_data  = list(colunas_data  or [])
    excluir_valor = list(excluir_valor or [])
    excluir_data  = list(excluir_data or [])
    forcar_valor  = list(forcar_valor or [])
    forcar_data   = list(forcar_data or [])

    # Candidatas por prefixo
    candidatas = [c for c in df_cols if (not limitar_ao_prefixo or c.startswith(prefix))]

    # Heurísticas
    palavras_valor = ["valor", "renda", "preco", "preço", "subsidio", "financi", "fgts", "itbi", "laudo"]
    palavras_data  = ["data", "nascimento", "envio", "assinatura", "vencimento", "cadastro", "alteracao", "atualizacao"]

    auto_valor = [c for c in candidatas if any(p in base_name(c).lower() for p in palavras_valor)]
    auto_data  = [c for c in candidatas if any(p in base_name(c).lower() for p in palavras_data)]

    # Montagem final (respeita excluir/forcar)
    colunas_valor = sorted(set(colunas_valor + auto_valor + forcar_valor) - set(excluir_valor))
    colunas_data  = sorted(set(colunas_data  + auto_data  + forcar_data)  - set(excluir_data))

    # Logs úteis
    print("🎯 Configuração aplicada:")
    print(f"   💰 Forçar valor: {', '.join(forcar_valor) if forcar_valor else 'Nenhuma'}")
    print(f"   📅 Forçar data:  {', '.join(forcar_data)  if forcar_data  else 'Nenhuma'}")
    print(f"   🚫 Excluir valor: {', '.join(excluir_valor) if excluir_valor else 'Nenhuma'}")
    print(f"   🚫 Excluir data:  {', '.join(excluir_data)  if excluir_data  else 'Nenhuma'}")
    print(f"   💰 Colunas finais de valor ({len(colunas_valor)}): {colunas_valor}")
    print(f"   📅 Colunas finais de data  ({len(colunas_data)}): {colunas_data}")

    # 1️⃣ IDs -> Integer (quando forem strings numéricas)
    for c in [x for x in df_cols if x.lower().startswith("id")]:
        if dtypes.get(c) == "string":
            df_tratado = df_tratado.withColumn(
                c,
                F.when(F.trim(F.col(c)).rlike(r"^[0-9]+$"), F.col(c).cast(T.IntegerType())).otherwise(F.col(c))
            )

    # 2️⃣ Valores -> Double
    for c in colunas_valor:
        if c in df_cols:
            s = F.col(c).cast("string")
            s = F.regexp_replace(s, r"(?i)\s*(r\$|rs|\$|€|£|:)", "")
            s = F.regexp_replace(s, r"\.", "")
            s = F.regexp_replace(s, r",", ".")
            s = F.regexp_replace(s, r"[^0-9\.\-\+]", "")
            s = F.trim(s)
            n = F.when(s.rlike(r"^[+-]?\d+(\.\d+)?$"), s.cast(T.DoubleType())).otherwise(None)
            if zero_um_como_nulo:
                n = F.when((n <= 1.0) | n.isNull(), None).otherwise(n)
            df_tratado = df_tratado.withColumn(c, n)

    # 3️⃣ Datas -> Timestamp
    def _parse_ts_string(scol):
        s = F.trim(scol.cast("string"))
        s = F.regexp_replace(s, r"[Tt]", " ")
        s = F.regexp_replace(s, r"\s*Z$", "")
        s = F.regexp_replace(s, r"(\d{2}:\d{2}:\d{2})\.\d+\s*$", r"\1")
        s = F.regexp_replace(s, r"/", "-")

        ts = F.coalesce(
            F.col(c).cast("timestamp"),
            F.to_timestamp(s, "yyyy-MM-dd HH:mm:ss"),
            F.to_timestamp(s, "yyyy-MM-dd"),
            F.to_timestamp(s, "dd-MM-yyyy HH:mm:ss"),
            F.to_timestamp(s, "dd-MM-yyyy"),
            F.to_timestamp(s, "MM-dd-yyyy HH:mm:ss"),
            F.to_timestamp(s, "MM-dd-yyyy")
        )
        ts = F.when((s.isNull()) | (s == "") | (F.lower(s) == "null"), None).otherwise(ts)
        ts = F.when(ts.isNull() | (F.year(ts) < 1900) | (F.year(ts) > 2100), None).otherwise(ts)
        return ts

    for c in colunas_data:
        if c in df_cols:
            df_tratado = df_tratado.withColumn(c, _parse_ts_string(F.col(c)))

    print("✅ Tratamento concluído — conversões aplicadas sem afetar colunas saudáveis da mesma linha.")
    return df_tratado


In [ ]:
from pyspark.sql import functions as F, types as T

def sanitize_for_sql_endpoint(df):
    """
    Garante compatibilidade com o SQL Endpoint do Fabric:
    - Em colunas string: "", " ", "null", "nan", "inf", "infinity" (qualquer caixa) -> NULL
    - Em colunas numéricas (Float/Double): NaN, +Inf, -Inf -> NULL
    Não altera tipos das colunas.
    """
    out = df

    # Strings "vazias" ou marcadores viram NULL
    STR_NULLS = ["", " ", "null", "NULL", "nan", "NaN", "inf", "Inf", "infinity", "Infinity"]
    for f in out.schema.fields:
        c = f.name
        if isinstance(f.dataType, T.StringType):
            out = out.withColumn(
                c,
                F.when(F.trim(F.col(c)).isin(STR_NULLS), None).otherwise(F.col(c))
            )

    # Floats/Doubles: NaN e ±Infinity viram NULL
    for f in out.schema.fields:
        c = f.name
        if isinstance(f.dataType, (T.FloatType, T.DoubleType)):
            out = out.withColumn(
                c,
                F.when(
                    F.isnan(F.col(c)) | (F.col(c) == float("inf")) | (F.col(c) == float("-inf")),
                    None
                ).otherwise(F.col(c))
            )

    return out


In [ ]:
# exclusões usando nomes *normalizados* (a função resolve para o nome real)
excluir_valor = ["aprovacao_vpl_valor"]
excluir_data  = ["status_registro", "juros_apos_entrega_cadastro", "idprecadastro", "juros_cadastro", "juros_cadastro_fixa_adicional", "vencimento"]

df_reserva_bronze = tratar_valores_e_datas(
    df_reserva_bronze,
    excluir_valor=excluir_valor,
    excluir_data=excluir_data,
    explicit_prefix=None,          # sem prefixo
    limitar_ao_prefixo=False,      # varre todas as colunas
    auto_prefix_listas=False,      # não tenta prefixar listas
    match_normalizado_nas_listas=True  # casa nomes normalizados com os reais
)

print("✅ RESERVA tratados e tipados com sucesso.")
df_reserva_bronze.printSchema()


In [ ]:
excluir_valor = ["possibilidade_venda", "inserido_bolsao", "score"]
excluir_data  = []

df_leads_bronze = tratar_valores_e_datas(
    df_leads_bronze,
    excluir_valor=excluir_valor,
    excluir_data=excluir_data,
    explicit_prefix=None,
    limitar_ao_prefixo=False,
    auto_prefix_listas=False,
    match_normalizado_nas_listas=True
)

print("✅ LEAD tratados e tipados com sucesso.")
df_leads_bronze.printSchema()


In [ ]:
excluir_valor = ["itbi_pago", "recebendo_financiamento", "data_laudo_liberado", "data_status_financiamento"]
excluir_data  = ["liberar_assinatura", "registro_pago", "valor_registro"]
colunas_data  = ["ultima_atualizacao", "data_laudo_liberado", "data_status_financiamento"]  # força tratamento dessa coluna como data

df_repasse_bronze = tratar_valores_e_datas(
    df_repasse_bronze,
    excluir_valor=excluir_valor,
    excluir_data=excluir_data,
    colunas_data=colunas_data,
    explicit_prefix=None,
    limitar_ao_prefixo=False,
    auto_prefix_listas=False,
    match_normalizado_nas_listas=True
)

print("✅ REPASSE tratados e tipados com sucesso.")
df_repasse_bronze.printSchema()


In [ ]:
excluir_valor = ["carta_credito"]
excluir_data  = ["vencimento", "idprecadastro", "sla_vencimento"]

df_precadastro_bronze = tratar_valores_e_datas(
    df_precadastro_bronze,
    excluir_valor=excluir_valor,
    excluir_data=excluir_data,
    explicit_prefix=None,
    limitar_ao_prefixo=False,
    auto_prefix_listas=False,
    match_normalizado_nas_listas=True
)

print("✅ PRECADASTRO tratados e tipados com sucesso.")
df_precadastro_bronze.printSchema()


In [ ]:
def preparar_para_prata(
    df,
    coluna_id: str,
    sistema_origem: str = "CVCRM",
    filtrar_ativo: bool = True
):
    """
    Função otimizada para uso em Lakehouse do Microsoft Fabric:
    🔹 Deduplicação eficiente por ID e data
    🔹 Conversão robusta de IDs múltiplos e strings vazias
    🔹 Redução de ações (menos collect/count)
    🔹 Ideal para uso em cluster com paralelismo controlado
    """

    # 🔒 Validação
    if "referencia_data" not in df.columns:
        raise ValueError("A coluna obrigatória 'referencia_data' não existe no DataFrame.")

    # 1️⃣ IDs válidos
    base = df.filter((F.col(coluna_id).isNotNull()) & (trim(F.col(coluna_id)) != ""))

    # 2️⃣ Filtro ativo (opcional)
    if filtrar_ativo and "ativo" in base.columns:
        base = base.filter((F.col("ativo").isNull()) | (F.col("ativo") == F.lit("S")))

    # 3️⃣ Deduplicação por ID — mais recente por referencia_data
    w = Window.partitionBy(coluna_id).orderBy(F.col("referencia_data").desc(), F.monotonically_increasing_id())
    base = (
        base.withColumn("rn", row_number().over(w))
            .filter(F.col("rn") == 1)
            .drop("rn")
    )

    # 4️⃣ Strings vazias → NULL (em lote, 1 select)
    exprs = [
        when(trim(F.col(c)) == "", F.lit(None)).otherwise(F.col(c)).alias(c)
        if t == "string" else F.col(c)
        for c, t in base.dtypes
    ]
    base = base.select(*exprs)

    # 5️⃣ Detectar colunas com múltiplos IDs (com vírgulas)
    colunas_virgula = [
        c for c, t in base.dtypes
        if t == "string" and c.lower().startswith("id") and base.filter(F.col(c).contains(",")).limit(1).count() > 0
    ]

    for c in colunas_virgula:
        print(f"⚙️ Coluna '{c}' contém vírgulas — expandindo automaticamente as posições.")

        # Split + máximo tamanho (1 ação collect)
        base = base.withColumn("_split", split(trim(F.col(c)), ","))
        max_pos = base.select(F.max(size(F.col("_split")))).first()[0] or 0

        # Criação das novas colunas dinamicamente
        for i in range(max_pos):
            pos_col = f"posicao_{i+1}_{c}"
            base = base.withColumn(
                pos_col,
                F.when(F.col("_split").getItem(i).rlike("^[0-9]+$"),
                       F.col("_split").getItem(i).cast("int")).otherwise(F.lit(None))
            )

        # Mantém o último ID (último elemento da lista)
        base = base.withColumn(
            c,
            F.when(element_at(F.col("_split"), -1).rlike("^[0-9]+$"),
                   element_at(F.col("_split"), -1).cast("int")).otherwise(F.lit(None))
        ).drop("_split")

    if colunas_virgula:
        print(f"✅ {len(colunas_virgula)} colunas de ID com múltiplos valores tratadas: {', '.join(colunas_virgula)}")
    else:
        print("✅ Nenhuma coluna de ID com múltiplos valores detectada.")

    # 6️⃣ Conversão em lote de colunas 'id' e 'posicao_' → int
    id_cols = [c for c in base.columns if c.lower().startswith(("id", "posicao_"))]
    for c in id_cols:
        base = base.withColumn(
            c,
            F.when(F.col(c).cast("int").isNotNull(), F.col(c).cast("int"))
             .otherwise(F.lit(None).cast("int"))
        )

    # 7️⃣ Metadados
    base = base.withColumn("data_processamento",F.from_utc_timestamp(F.current_timestamp(), "America/Sao_Paulo"))

    # 8️⃣ Reparticionamento lógico (por ID)
    base = base.repartition(F.col(coluna_id))

    print("✅ Função 'preparar_para_prata' concluída com sucesso.")
    return base


In [ ]:
def upsert_prata(
    df_final,
    path_prata: str,
    coluna_id: str,
    zorder_cols: list = None
):
    """
    PRATA:
    - Se df_final tiver `referencia` e `referencia_data`: MERGE incremental por ID + comparação de data.
    - Se NÃO tiver `referencia`/`referencia_data`: FULL REFRESH.
    - Cria PRATA se não existir.
    - Ajusta propriedades Delta e OPTIMIZE/ZORDER.
    - ⚠️ Se o DataFrame estiver vazio, não faz nada.
    - 🚫 Deduplica automaticamente o df_final pelo ID e maior referencia_data.
    """
    from delta.tables import DeltaTable
    from pyspark.sql import functions as F, Window

    if df_final is None or df_final.rdd.isEmpty():
        print(f"ℹ️ Nenhum registro novo encontrado. Nenhuma ação realizada para {path_prata}.")
        return

    cols_l = [c.lower() for c in df_final.columns]
    tem_ref = ("referencia" in cols_l) and ("referencia_data" in cols_l)

    spark.conf.set("spark.databricks.delta.schema.autoMerge.enabled", "true")

    # 🔹 Deduplicar se tiver referencia_data
    if tem_ref:
        window = Window.partitionBy(coluna_id).orderBy(F.desc("referencia_data"))
        df_final = (
            df_final.withColumn("rn", F.row_number().over(window))
            .filter(F.col("rn") == 1)
            .drop("rn")
        )
        print(f"🧹 {df_final.count()} registros após deduplicação por {coluna_id} e referencia_data")

    # PRATA não existe → criar (overwrite)
    if not DeltaTable.isDeltaTable(spark, path_prata):
        print(f"🆕 Criando PRATA em {path_prata}")
        (
            df_final
            .coalesce(min(8, max(1, df_final.rdd.getNumPartitions())))
            .write.format("delta")
            .mode("overwrite")
            .option("overwriteSchema", "true")
            .save(path_prata)
        )
    else:
        if not tem_ref:
            print(f"🧹 FULL REFRESH da PRATA (sem referencia_*): sobrescrevendo {path_prata}")
            (
                df_final
                .coalesce(min(8, max(1, df_final.rdd.getNumPartitions())))
                .write.format("delta")
                .mode("overwrite")
                .option("overwriteSchema", "true")
                .save(path_prata)
            )
        else:
            print(f"🔁 MERGE incremental em {path_prata}")
            t = DeltaTable.forPath(spark, path_prata)
            merge_condition = f"t.{coluna_id} = n.{coluna_id}"

            (
                t.alias("t")
                .merge(df_final.alias("n"), merge_condition)
                .whenMatchedUpdateAll(condition="t.referencia_data IS NULL OR n.referencia_data > t.referencia_data")
                .whenNotMatchedInsertAll(condition="n.referencia IS NOT NULL AND n.referencia_data IS NOT NULL")
                .execute()
            )

    # Propriedades Delta e otimização
    spark.sql(f"""
        ALTER TABLE delta.`{path_prata}`
        SET TBLPROPERTIES (
            'delta.enableChangeDataFeed' = 'true',
            'delta.autoOptimize.optimizeWrite' = 'true',
            'delta.autoOptimize.autoCompact'  = 'true'
        )
    """)

    zcols = zorder_cols or [coluna_id]
    print(f"🧩 Otimizando tabela com ZORDER BY ({', '.join(zcols)}) ...")
    spark.sql(f"OPTIMIZE delta.`{path_prata}` ZORDER BY ({', '.join(zcols)})")

    print(f"✅ PRATA atualizada e otimizada em {path_prata}")


In [ ]:
def verificar_ids_com_virgula(df, nome_tabela: str, limite_exemplo: int = 50):
    """
    🔍 Verifica colunas que começam com 'id' e detecta registros com múltiplos IDs separados por vírgula.
    ✅ Versão otimizada para Spark / Fabric (executa em apenas 1 job principal)
    
    Parâmetros:
        df (DataFrame): DataFrame a ser analisado
        nome_tabela (str): Nome lógico da tabela (para exibição no log)
        limite_exemplo (int): Quantidade de exemplos para exibição (default=50)
    """
    print(f"\n=== {nome_tabela.upper()} ===")

    # 🔹 1️⃣ Seleciona apenas colunas que começam com 'id'
    colunas_id = [c for c in df.columns if c.lower().startswith("id")]

    if not colunas_id:
        print("⚠️ Nenhuma coluna iniciando com 'id' encontrada.")
        return

    print(f"🔎 Colunas de ID analisadas: {', '.join(colunas_id)}")

    # 🔹 2️⃣ Cria expressão única para todas as colunas com vírgula
    condicao_virgula = F.lit(False)
    for c in colunas_id:
        condicao_virgula = condicao_virgula | F.col(c).cast("string").contains(",")

    df_com_virgula = df.filter(condicao_virgula)

    # 🔹 3️⃣ Agrega contagem por coluna de forma VETORIZADA (sem múltiplos .count())
    exprs_contagem = [
        F.sum(F.when(F.col(c).cast("string").contains(","), 1).otherwise(0)).alias(c)
        for c in colunas_id
    ]
    df_contagens = df.agg(*exprs_contagem).first().asDict()

    # 🔹 4️⃣ Exibe resultados
    encontrou = False
    print("\n📊 Colunas com vírgula:")
    for c, qtd in df_contagens.items():
        if qtd and qtd > 0:
            encontrou = True
            print(f"  • {c}: {qtd:,} registros com vírgula")

    if not encontrou:
        print("✅ Nenhuma coluna de ID contém vírgula.")
        return

    # 🔹 5️⃣ Exibe exemplos distintos (limitado)
    print(f"\n🧾 Exemplo de registros com vírgula (até {limite_exemplo}):")
    df_com_virgula.select(*colunas_id).distinct().limit(limite_exemplo).show(truncate=False)


In [ ]:
verificar_ids_com_virgula(df_reserva_bronze, "RESERVA")

In [ ]:
verificar_ids_com_virgula(df_leads_bronze, "LEAD")


In [ ]:
verificar_ids_com_virgula(df_repasse_bronze, "REPASSE")


In [ ]:
verificar_ids_com_virgula(df_precadastro_bronze, "PRECADASTRO")

##### [Inserir Lead]

In [ ]:
# Ex.: LEAD
inicio = time.perf_counter()
df_leads_prata = preparar_para_prata(df_leads_bronze, coluna_id="idlead", sistema_origem="CVCRM")
print(f"🕐 Tempo preparar_para_prata: {time.perf_counter() - inicio:.2f}s")


In [ ]:
# ✅ Sanitiza antes do upsert
df_leads_prata = sanitize_for_sql_endpoint(df_leads_prata)

In [ ]:
df_leads_prata = df_leads_prata.persist()
# ou .cache() se não usar StorageLevel

In [ ]:
df_leads_prata = df_leads_prata.repartition(4, F.col("idlead"))

In [ ]:
inicio = time.perf_counter()
upsert_prata(df_leads_prata, f"{prata_lakehouse_path}/lead_prata", coluna_id="idlead", zorder_cols=["idlead","referencia_data"])
print(f"🕐 Tempo upsert_prata: {time.perf_counter() - inicio:.2f}s")

In [ ]:
spark.sql(f"OPTIMIZE delta.`{prata_lakehouse_path}/lead_prata` ZORDER BY (idlead)")
spark.sql(f"VACUUM delta.`{prata_lakehouse_path}/lead_prata` RETAIN 168  HOURS")  # mantém 7 dias

##### [Inserir Precadastro]

In [ ]:
inicio = time.perf_counter()
df_precadastro_prata = preparar_para_prata(
    df_precadastro_bronze,
    coluna_id="idprecadastro",
    sistema_origem="CVCRM"
)
print(f"🕐 Tempo preparar_para_prata (precadastro): {time.perf_counter() - inicio:.2f}s")


In [ ]:
# ✅ Sanitiza antes do upsert
df_precadastro_prata = sanitize_for_sql_endpoint(df_precadastro_prata)

In [ ]:
# 🔹 Persist para evitar reprocessamento
df_precadastro_prata = df_precadastro_prata.persist()
# 🔹 Particionamento controlado — base moderada
df_precadastro_prata = df_precadastro_prata.repartition(4, F.col("idprecadastro"))


In [ ]:
# 🔹 Upsert otimizado
inicio = time.perf_counter()
upsert_prata(
    df_precadastro_prata,
    f"{prata_lakehouse_path}/precadastro_prata",
    coluna_id="idprecadastro",
    zorder_cols=["idprecadastro", "referencia_data"]
)
print(f"🕐 Tempo upsert_prata (precadastro): {time.perf_counter() - inicio:.2f}s")

In [ ]:
# 🔹 Compactação e limpeza
spark.sql(f"OPTIMIZE delta.`{prata_lakehouse_path}/precadastro_prata` ZORDER BY (idprecadastro)")
spark.sql(f"VACUUM delta.`{prata_lakehouse_path}/precadastro_prata` RETAIN 168 HOURS")

##### [Inserir Reserva]

In [ ]:
# ------------------- RESERVA -------------------
inicio = time.perf_counter()
df_reserva_prata = preparar_para_prata(
    df_reserva_bronze,
    coluna_id="idreserva",
    sistema_origem="CVCRM"
)
print(f"🕐 Tempo preparar_para_prata (reserva): {time.perf_counter() - inicio:.2f}s")


In [ ]:
# ✅ Sanitiza antes do upsert
df_reserva_prata = sanitize_for_sql_endpoint(df_reserva_prata)

In [ ]:
df_reserva_prata = df_reserva_prata.persist()

# 🔹 Base maior → mais partições
df_reserva_prata = df_reserva_prata.repartition(8, F.col("idreserva"))


In [ ]:
df_reserva_prata.printSchema()


In [ ]:
inicio = time.perf_counter()
upsert_prata(
    df_reserva_prata,
    f"{prata_lakehouse_path}/reserva_prata",
    coluna_id="idreserva",
    zorder_cols=["idreserva", "referencia_data"]
)
print(f"🕐 Tempo upsert_prata (reserva): {time.perf_counter() - inicio:.2f}s")


In [ ]:
spark.sql(f"OPTIMIZE delta.`{prata_lakehouse_path}/reserva_prata` ZORDER BY (idreserva)")
spark.sql(f"VACUUM delta.`{prata_lakehouse_path}/reserva_prata` RETAIN 168 HOURS")

###### [Reserva]

In [ ]:
# ------------------- REPASSE -------------------
inicio = time.perf_counter()
df_repasse_prata = preparar_para_prata(
    df_repasse_bronze,
    coluna_id="idrepasse",
    sistema_origem="CVCRM"
)
print(f"🕐 Tempo preparar_para_prata (repasse): {time.perf_counter() - inicio:.2f}s")


In [ ]:
# ✅ Sanitiza antes do upsert
df_repasse_prata = sanitize_for_sql_endpoint(df_repasse_prata)

In [ ]:
df_repasse_prata = df_repasse_prata.persist()

# 🔹 Base geralmente menor → menos partições
df_repasse_prata = df_repasse_prata.repartition(4, F.col("idrepasse"))


In [ ]:
inicio = time.perf_counter()
upsert_prata(
    df_repasse_prata,
    f"{prata_lakehouse_path}/repasse_prata",
    coluna_id="idrepasse",
    zorder_cols=["idrepasse", "referencia_data"]
)
print(f"🕐 Tempo upsert_prata (repasse): {time.perf_counter() - inicio:.2f}s")


In [ ]:
spark.sql(f"OPTIMIZE delta.`{prata_lakehouse_path}/repasse_prata` ZORDER BY (idrepasse)")
spark.sql(f"VACUUM delta.`{prata_lakehouse_path}/repasse_prata` RETAIN 168 HOURS")

## [**Tratar Campos Adicionais Tratamento**]

In [ ]:
path_campos_bronze = f"{bronze_lakehouse_path}/campos_adicionais_bronze_cvdw"
path_campos_prata  = f"{prata_lakehouse_path}/campos_adicionais_prata"

In [ ]:
spark.conf.set("spark.sql.legacy.timeParserPolicy", "LEGACY")
spark.conf.set("spark.sql.parquet.datetimeRebaseModeInWrite", "CORRECTED")
spark.conf.set("spark.sql.parquet.int96RebaseModeInWrite", "CORRECTED")
spark.conf.set("spark.sql.parquet.outputTimestampType", "TIMESTAMP_MICROS")


In [ ]:
EXCLUSOES_CAMPOS = {
    "lead": {
        "excluir_valor": [
            "lead_categoria_de_renda_parceiro_conjuge",
            "lead_categoria_de_renda",
            "lead_descricao_subsidio_price",
            "lead_declarou_imposto_de_renda",
            "lead_diferenca_da_renta_atual_para_a_renda_ideal_price",
            "lead_diferenca_da_renta_atual_para_a_renda_ideal_sac",
            "lead_link_resultados_price_renda_ideal",
            "lead_link_resultados_sac_renda_ideal",
            "lead_modalidade_de_financiamento_simulacao",
            "lead_descricao_subsidio_sac",
            "lead_possui_3_anos_sob_regime_do_fgts",
            "lead_renda_ideal_r_i_para_price",
            "lead_renda_ideal_r_i_para_sac",
            "lead_renda_parceiro_conjuge",
            "lead_possui_fgts",
            "lead_quantos_anos_de_fgts",
            "lead_renda_principal",
            "lead_faixa_de_valor"
        ],
        "forcar_valor": [
            "lead_restricao_lead"
        ],
        "excluir_data": []
    },

    "precadastro": {
        "excluir_valor": [
            "precadastro_3_anos_de_fgts",
            "precadastro_avaliacao_de_risco_com_quantos_participantes_com_renda",
            "precadastro_categoria_de_renda",
            "precadastro_modalidade_de_financiamento",
            "precadastro_possui_3_anos_sob_regime_do_fgts",
            "precadastro_tipo_da_renda"
        ],
        "forcar_valor": [
            "precadastro_valor_de_financiamento_apenas_fi",
            "precadastro_valor_de_laudo_do_imovel_pretendido",
            "precadastro_valor_do_subsidio_concedido_cef",
            "precadastro_valor_itbi",
            "precadastro_renda_bruta",
            "precadastro_renda_bruta_formal",
            "precadastro_renda_informal",
            "precadastro_renda_liquida"
        ],
        "forcar_data": [],
        "excluir_data": [
            "precadastro_motivo_da_analise",
            "precadastro_profissao_da_analise_de_credito",
            "precadastro_resposta_do_perfil",
            "precadastro_resultado_da_analise",
            "precadastro_3_anos_de_fgts",
            "precadastro_aguardando_qv",
            "precadastro_avaliacao_de_risco_com_quantos_participantes_com_renda",
            "precadastro_cadastro_bloqueado",
            "precadastro_cadastro_irregular_na_receita",
            "precadastro_categoria_de_renda",
            "precadastro_documentacao_pendente",
            "precadastro_e_seu_primeiro_imovel",
            "precadastro_erro_cadastral",
            "precadastro_estado_civil",
            "precadastro_formacao",
            "precadastro_justificativa_nao_venda",
            "precadastro_modalidade_de_financiamento",
            "precadastro_momento_do_pre_cadastro",
            "precadastro_numero_de_qv_disponivel",
            "precadastro_observacao",
            "precadastro_onde_reside",
            "precadastro_parcela_aprovada",
            "precadastro_pasta_duplicada",
            "precadastro_possibilidade_de_venda",
            "precadastro_possui_3_anos_sob_regime_do_fgts",
            "precadastro_possui_dependente",
            "precadastro_processo_a_intenalizar_agencia",
            "precadastro_processo_intenalizado_agencia",
            "precadastro_profissao_1",
            "precadastro_restricao_1",
            "precadastro_restricao_2",
            "precadastro_sexo",
            "precadastro_status_da_negociacao",
            "precadastro_sub_fase",
            "precadastro_tipo_da_renda",
            "precadastro_tipo_de_dependente",
            'precadastro_desistente', 'precadastro_renda_bruta', 'precadastro_renda_bruta_formal', 'precadastro_renda_informal', 'precadastro_renda_liquida', 'precadastro_valor_de_financiamento_apenas_fi', 'precadastro_valor_de_laudo_do_imovel_pretendido', 'precadastro_valor_do_subsidio_concedido_cef', 'precadastro_valor_itbi',
            "precadastro_profissao_da_analise_de_credito"
        ]
    },

    "reserva": {
        "excluir_valor": ["reserva_categoria_de_renda_do_fiador"],
        "excluir_data": ["reserva_data_do_kit_registro_ok"]
    },

    "repasse": {
        "excluir_valor": [
            "repasse_ap_itbi",
            "repasse_fgts_futuro",
            "repasse_protocolo_itbi",
            "repasse_valor_do_contrato_7lm",
            "repasse_valor_do_contrato_caixa",
            "repasse_valor_do_produto_caixa",
            "repasse_valor_itbi",
            "repasse_classificacao_preco"
        ],
        "excluir_data": [
            "repasse_data_de_solicitacao_do_itbi_prefeitura",
            "repasse_data_pagamento_do_itbi",
            "repasse_data_solicitacao_pagamento_itbi_financeiro",
            "repasse_pendencia_de_repasse_descricao_e_data",
            "repasse_previsao_de_data_de_saida_do_cartorio",
            "repasse_probabilidade_de_assinatura"
        ]
    },

    "distrato": {"excluir_valor": [], "excluir_data": []}
}

CHAVES_GRUPO = ["idfuncionalidade", "funcionalidade", "referencia", "referencia_data"]

COLLISION_OVERRIDES = {
    "lead": {
        "Primeiro Imóvel?": "primeiro_imovel_1",
        "primeiro_imovel": "primeiro_imovel_2",
        "Região de Interesse": "regiao_de_interesse_1",
        "Região de interesse": "regiao_de_interesse_2",
    },
    "precadastro": {
        "Restrição": "restricao_1",
        "Restrição?": "restricao_2",
    },
    "repasse": {
        "Obs Garantia": "obs_garantia_1",
        "Obs Garantia:": "obs_garantia_2",
    },
}

PATHS_MAIN_PRATA = {
    "lead":        f"{prata_lakehouse_path}/lead_prata",
    "precadastro": f"{prata_lakehouse_path}/precadastro_prata",
    "repasse":     f"{prata_lakehouse_path}/repasse_prata",
    "reserva":     f"{prata_lakehouse_path}/reserva_prata",
}


In [ ]:
def _norm_value(v: str):
    """Normaliza valores textuais, removendo aspas externas, estruturas serializadas e strings vazias."""
    if v is None:
        return None

    v = str(v).strip()

    # Remove aspas externas diretas: "SIM" → SIM
    if len(v) >= 2 and v.startswith('"') and v.endswith('"'):
        v = v[1:-1].strip()

    # Captura o último valor entre aspas dentro de estruturas serializadas
    # Ex: a:1:{s:0:"SIM";} -> SIM
    m = re.findall(r'"([^"]+)"', v)
    if m:
        return m[-1].strip()

    # Se estiver vazio, retorna None
    if v.strip() == "":
        return None

    return v

def _norm_basic(s: str) -> str:
    """Normaliza nomes removendo acentos, aspas, espaços e caracteres especiais."""
    s = str(s).replace('"', '').replace("'", "")
    s2 = unicodedata.normalize("NFKD", s).encode("ascii", "ignore").decode("ascii")
    s2 = re.sub(r"[^A-Za-z0-9_]+", "_", s2.strip().lower())
    s2 = re.sub(r"_+", "_", s2).strip("_")
    return s2 or "col"



def fix_duplicate_columns(df, label: str = ""):
    """Corrige nomes de colunas duplicadas dentro do mesmo DF."""
    cols = df.columns
    cont = Counter(cols)

    if all(v == 1 for v in cont.values()):
        print(f"✅ [{label}] Nenhuma coluna duplicada interna para corrigir.")
        return df

    seen = {}
    for c in cols:
        seen[c] = seen.get(c, 0) + 1
        if seen[c] > 1:
            new_name = f"{c}__fix{seen[c]}"
            print(f"   - Renomeado: {c} -> {new_name}")
            df = df.withColumnRenamed(c, new_name)

    return df



In [ ]:
def _build_safe_mapping_for_pivot(df_func, func_label):
    """Gera mapeamento seguro de campo_nome → campo_nome_safe considerando overrides."""
    raw_vals = [r[0] for r in df_func.select("campo_nome").distinct().collect()]
    overrides = COLLISION_OVERRIDES.get(func_label, {})

    used = defaultdict(int)
    mapping_rows, listed_overrides = [], []

    # aplica overrides
    for rv in raw_vals:
        if rv in overrides:
            mapping_rows.append((rv, overrides[rv]))
            listed_overrides.append(rv)

    # normalização padrão
    for rv in raw_vals:
        if rv in listed_overrides:
            continue

        rv_clean = str(rv).replace('"', '').replace("'", "").strip()
        base = _norm_basic(rv_clean)

        used[base] += 1
        idx = used[base]
        safe = base if idx == 1 else f"{base}__pvdup{idx}"

        mapping_rows.append((rv, safe))

    schema = T.StructType([
        T.StructField("campo_nome", T.StringType(), True),
        T.StructField("campo_nome_safe", T.StringType(), True),
    ])

    return spark.createDataFrame(mapping_rows, schema)


In [ ]:
def _pivotar_com_prefixo_debug(df_bronze_inc):
    """Pivotação segura, prefixos e consolidação por funcionalidade."""

    # ===============================================================
    # 1️⃣ NORMALIZAÇÃO DE VALORES (ANTES DO PIVOT)  <<<< ESTE É O PASSO 5
    # ===============================================================
    df_bronze_inc = df_bronze_inc.withColumn(
        "valor", F.udf(_norm_value, T.StringType())(F.col("valor"))
    )
    print("🧹 Valores normalizados antes do pivot.")

    # ===============================================================
    # Identifica funcionalidades existentes
    # ===============================================================
    funcionalidades = (
        df_bronze_inc.select("funcionalidade")
        .distinct()
        .rdd.flatMap(lambda x: x)
        .collect()
    )

    dfs_tratados = []

    # ===============================================================
    # Processa cada funcionalidade isoladamente
    # ===============================================================
    for func in funcionalidades:
        print(f"📊 Pivotando funcionalidade: {func}")

        df_func = df_bronze_inc.filter(F.col("funcionalidade") == func)

        # ===============================================================
        # 2️⃣ Gera mapeamento seguro de nomes de campo
        # ===============================================================
        map_df = _build_safe_mapping_for_pivot(df_func, func)

        df_func_safe = (
            df_func.join(F.broadcast(map_df), on="campo_nome", how="left")
                   .withColumn("campo_nome_safe",
                               F.coalesce(F.col("campo_nome_safe"), F.lit("col")))
        )

        # ===============================================================
        # 3️⃣ Pivot real
        # ===============================================================
        df_f = (
            df_func_safe
              .groupBy(*CHAVES_GRUPO)
              .pivot("campo_nome_safe")
              .agg(F.first("valor"))
        )

        # ===============================================================
        # 4️⃣ Prefixa colunas baseadas na funcionalidade
        # ===============================================================
        chaves_set = set(CHAVES_GRUPO)
        alias_exprs = [F.col(k) for k in CHAVES_GRUPO]

        for c in [c for c in df_f.columns if c not in chaves_set]:
            alias_exprs.append(F.col(f"`{c}`").alias(f"{func}_{c}"))

        df_alias = df_f.select(*alias_exprs)

        # ===============================================================
        # 5️⃣ Remove colunas duplicadas internas
        # ===============================================================
        df_clean = fix_duplicate_columns(df_alias, label=func)

        # ===============================================================
        # 6️⃣ Consolidação final por idfuncionalidade + funcionalidade
        # ===============================================================
        id_col = "idfuncionalidade"

        non_key_cols = [c for c in df_clean.columns if c not in CHAVES_GRUPO]

        agg_exprs = [
            F.first(F.when(F.col(c).isNotNull(), F.col(c)), ignorenulls=True).alias(c)
            for c in non_key_cols
        ]

        df_consolidado = (
            df_clean
            .groupBy(id_col, "funcionalidade")
            .agg(
                F.max("referencia").alias("referencia"),
                F.max("referencia_data").alias("referencia_data"),
                *agg_exprs
            )
        )

        dfs_tratados.append(df_consolidado)

    # ===============================================================
    # 7️⃣ União final com alinhamento de colunas
    # ===============================================================
    all_cols = sorted(set().union(*[set(d.columns) for d in dfs_tratados]))

    ordered_cols = (
        [c for c in CHAVES_GRUPO if c in all_cols] +
        [c for c in all_cols if c not in CHAVES_GRUPO]
    )

    def _sel(d): 
        return [c for c in ordered_cols if c in d.columns]

    df_final = dfs_tratados[0].select(_sel(dfs_tratados[0]))

    for d in dfs_tratados[1:]:
        df_final = df_final.unionByName(d.select(_sel(d)), allowMissingColumns=True)

    print(f"✅ Pivotação concluída e consolidada: {df_final.count()} linhas / {len(df_final.columns)} colunas.")
    return df_final, None


In [ ]:
def ler_incremental_bronze_campos(path_bronze: str, path_prata: str, funcionalidade: str):
    """
    Retorna (df_bronze_inc, full_load: bool, ultima_ref).
    Faz leitura incremental da Bronze considerando a última referencia_data
    da FUNCIONALIDADE específica já existente na PRATA (campos adicionais).
    """
    print(f"\n🔎 Lendo Bronze de forma incremental... (funcionalidade: {funcionalidade})")
    spark.conf.set("spark.databricks.delta.schema.autoMerge.enabled", "true")

    # Se PRATA (campos) não existe → full load daquela funcionalidade
    if not DeltaTable.isDeltaTable(spark, path_prata):
        print("🆕 Nenhuma PRATA de Campos Adicionais — carga completa da Bronze.")
        df_bronze_inc = (
            spark.read.format("delta").load(path_bronze)
                 .filter(F.col("funcionalidade") == funcionalidade)
        )
        return df_bronze_inc, True, None

    # Última referência já processada na PRATA (campos) para a funcionalidade
    df_prata = spark.read.format("delta").load(path_prata)
    ultima_ref_row = (
        df_prata.filter(F.col("funcionalidade") == funcionalidade)
                .select(F.max("referencia_data").alias("ultima_ref"))
                .first()
    )
    ultima_ref = ultima_ref_row["ultima_ref"] if ultima_ref_row else None
    print(f"📅 Última referencia_data (na PRATA - {funcionalidade}): {ultima_ref}")

    # Se não tem histórico na PRATA → full load
    if not ultima_ref:
        print(f"🆕 Nenhuma referência anterior para {funcionalidade} — carga completa.")
        df_bronze_inc = (
            spark.read.format("delta").load(path_bronze)
                 .filter(F.col("funcionalidade") == funcionalidade)
        )
        return df_bronze_inc, True, None

    # Incremental via CDF (fallback por referencia_data)
    try:
        df_bronze_inc = (
            spark.read.format("delta")
                 .option("readChangeFeed", "true")
                 .option("startingTimestamp", ultima_ref)
                 .load(path_bronze)
                 .filter(
                     (F.col("_change_type").isin("insert", "update_postimage")) &
                     (F.col("funcionalidade") == funcionalidade)
                 )
                 .drop("_change_type", "_commit_version", "_commit_timestamp")
        )
        print(f"⚡ Incremental via Change Data Feed (CDF) para {funcionalidade}.")
    except Exception as e:
        print(f"⚠️ CDF indisponível — fallback por referencia_data. ({e})")
        df_bronze_inc = (
            spark.read.format("delta").load(path_bronze)
                 .filter(
                     (F.col("referencia_data") > F.lit(ultima_ref)) &
                     (F.col("funcionalidade") == funcionalidade)
                 )
        )
    return df_bronze_inc, False, ultima_ref

In [ ]:
def _normalizar_exclusoes(funcionalidade: str, cols_presentes: list):
    """
    Identifica colunas que NÃO devem ser tratadas como valor/data
    (mas permanecem no DF final).
    """
    cfg = EXCLUSOES_CAMPOS.get(funcionalidade, {})
    excluir_valor = set(cfg.get("excluir_valor", []))
    excluir_data  = set(cfg.get("excluir_data", []))
    forcar_valor  = set(cfg.get("forcar_valor", []))
    forcar_data   = set(cfg.get("forcar_data", []))

    existentes = set(cols_presentes)
    validas_excluir_valor = sorted(list(excluir_valor & existentes))
    validas_excluir_data  = sorted(list(excluir_data  & existentes))
    validas_forcar_valor  = sorted(list(forcar_valor & existentes))
    validas_forcar_data   = sorted(list(forcar_data  & existentes))

    ausentes = sorted(list((excluir_valor | excluir_data | forcar_valor | forcar_data) - existentes))
    if ausentes:
        print(f"ℹ️ Colunas configuradas mas ausentes no schema ({funcionalidade}): {', '.join(ausentes)}")

    return {
        "excluir_valor": validas_excluir_valor,  # NÃO tratar como valor
        "excluir_data":  validas_excluir_data,   # NÃO tratar como data
        "forcar_valor":  validas_forcar_valor,
        "forcar_data":   validas_forcar_data,
    }

def _cast_valor(col):
    """
    Converte strings tipo BR para número:
    - remove 'R$' e texto
    - remove separador de milhar '.'
    - converte vírgula para ponto
    - parênteses = negativo
    """
    s = F.col(col).cast("string")
    sign = F.when(s.rlike(r"\("), F.lit(-1.0)) \
            .when(s.startswith("-"), -1.0) \
            .otherwise(1.0)
    cleaned = F.regexp_replace(s, r"[^\d,.\-()]", "")
    cleaned = F.regexp_replace(cleaned, r"\.", "")
    cleaned = F.regexp_replace(cleaned, r",", ".")
    num = cleaned.cast(T.DoubleType())
    return (sign * num)

def _cast_timestamp_multi(col):
    """Tenta múltiplos formatos comuns (BR e ISO)."""
    c = F.col(col).cast("string")
    return F.coalesce(
        F.to_timestamp(c, "yyyy-MM-dd HH:mm:ss"),
        F.to_timestamp(c, "yyyy-MM-dd'T'HH:mm:ss"),
        F.to_timestamp(c, "dd/MM/yyyy HH:mm:ss"),
        F.to_timestamp(c, "dd/MM/yyyy"),
        F.to_timestamp(c, "yyyy-MM-dd")
    )

In [ ]:
def tratar_funcionalidade(df_pivot, funcionalidade: str):
    f = funcionalidade
    prefix = f + "_"

    print("\n" + "=" * 100)
    print(f"🧩 INICIANDO TRATAMENTO: {f.upper()}")
    print("=" * 100)

    # mantém TODAS as colunas da funcionalidade + chaves
    cols_func = [c for c in df_pivot.columns if c.startswith(prefix)] + [c for c in CHAVES_GRUPO if c in df_pivot.columns]
    df_f = df_pivot.select(*cols_func).filter(F.col("funcionalidade") == f)

    print(f"📊 Total de colunas encontradas ({f}): {len(df_f.columns)}")
    print(f"🔑 Chaves presentes: {[k for k in CHAVES_GRUPO if k in df_f.columns]}")

    # 1) lê as listas de exceção/força e valida com schema
    cfg_ex = _normalizar_exclusoes(f, df_f.columns)
    colunas_excluir_valor = cfg_ex["excluir_valor"]  # NÃO tratar como valor
    colunas_excluir_data  = cfg_ex["excluir_data"]   # NÃO tratar como data
    colunas_forcar_valor  = cfg_ex["forcar_valor"]
    colunas_forcar_data   = cfg_ex["forcar_data"]

    print(f"🚫 NÃO tratar como VALOR: {', '.join(colunas_excluir_valor) if colunas_excluir_valor else '—'}")
    print(f"🚫 NÃO tratar como DATA:  {', '.join(colunas_excluir_data)  if colunas_excluir_data  else '—'}")
    print(f"💪 Forçar VALOR: {', '.join(colunas_forcar_valor) if colunas_forcar_valor else '—'}")
    print(f"📅 Forçar DATA:  {', '.join(colunas_forcar_data)  if colunas_forcar_data  else '—'}")

    # ❗ Não removemos nada: tratamos por cima de df_f
    df_para_tratar = df_f

    # 2) detecção automática de candidatos (pelo nome da coluna sem o prefixo)
    palavras_valor = ["valor", "renda", "preco", "preço", "subsidio", "financiamento", "fgts", "itbi", "laudo", "price", "sac"]
    palavras_data  = ["data", "nascimento", "envio", "assinatura", "vencimento", "cadastro", "alteracao", "atualizacao", "emissao"]

    def strip_prefix(c: str):
        return c[len(prefix):] if c.startswith(prefix) else c

    colunas_valor_auto = [c for c in df_para_tratar.columns if any(p in strip_prefix(c).lower() for p in palavras_valor)]
    colunas_data_auto  = [c for c in df_para_tratar.columns if any(p in strip_prefix(c).lower() for p in palavras_data)]

    # 3) combina: (auto ∪ forçar) − excluir  → listas finais a tratar
    colunas_valor = sorted( (set(colunas_valor_auto) | set(colunas_forcar_valor)) - set(colunas_excluir_valor) )
    colunas_data  = sorted( (set(colunas_data_auto)  | set(colunas_forcar_data))  - set(colunas_excluir_data) )

    print(f"💰 Tratamento VALOR ({len(colunas_valor)}): {', '.join(colunas_valor) if colunas_valor else '—'}")
    print(f"📅 Tratamento DATA  ({len(colunas_data)}): {', '.join(colunas_data)  if colunas_data  else '—'}")

    # 4) chama o tratador passando as listas de NÃO-TRATAR
    df_tratado = tratar_valores_e_datas(
        df_para_tratar,
        colunas_valor=colunas_valor,          # já SEM as excluídas
        colunas_data=colunas_data,            # já SEM as excluídas
        excluir_valor=colunas_excluir_valor,  # mantém no DF, só não trata
        excluir_data=colunas_excluir_data,    # mantém no DF, só não trata
        forcar_valor=colunas_forcar_valor,
        forcar_data=colunas_forcar_data,
        explicit_prefix=prefix,
        limitar_ao_prefixo=True,
        auto_prefix_listas=True,
        match_normalizado_nas_listas=True,
    )

    # 5) renomeia/casta ID e mantém último por id+funcionalidade
    id_col = f"id{f}"
    if id_col in df_tratado.columns:
        df_tratado = df_tratado.withColumn(id_col, F.col(id_col).cast(T.IntegerType()))
    elif "idfuncionalidade" in df_tratado.columns:
        df_tratado = df_tratado.withColumnRenamed("idfuncionalidade", id_col)

    if all(k in df_tratado.columns for k in [id_col, "funcionalidade", "referencia_data"]):
        w = Window.partitionBy(id_col, "funcionalidade").orderBy(F.col("referencia_data").desc())
        df_tratado = df_tratado.withColumn("rn", F.row_number().over(w)).filter("rn = 1").drop("rn")

    df_tratado = df_tratado.withColumn("data_processamento",F.from_utc_timestamp(F.current_timestamp(), "America/Sao_Paulo"))
    print(f"✅ Tratamento concluído ({f}) — {df_tratado.count()} linhas")
    return df_tratado

In [ ]:
def upsert_campos_adicionais_prata(df_campos_tratado, path_campos_prata: str):
    # Tabela não existe? cria
    if not DeltaTable.isDeltaTable(spark, path_campos_prata):
        (df_campos_tratado.write
            .format("delta")
            .mode("overwrite")
            .option("overwriteSchema", "true")
            .option("mergeSchema", "true")
            .save(path_campos_prata))
        spark.sql(f"""
          ALTER TABLE delta.`{path_campos_prata}`
          SET TBLPROPERTIES (
            'delta.enableChangeDataFeed'='true',
            'delta.autoOptimize.optimizeWrite'='true',
            'delta.autoOptimize.autoCompact'='true'
          )
        """)
        print(f"🆕 Criada PRATA de Campos Adicionais: {path_campos_prata}")
        return

    # Merge incremental
    cond = "t.idfuncionalidade = n.idfuncionalidade AND t.funcionalidade = n.funcionalidade"
    (DeltaTable.forPath(spark, path_campos_prata).alias("t")
        .merge(df_campos_tratado.alias("n"), cond)
        .whenMatchedUpdateAll(condition="n.referencia_data > t.referencia_data")
        .whenNotMatchedInsertAll()
        .execute())

    print("✅ Merge concluído em CAMPOS_ADICIONAIS_PRATA")


In [ ]:
def upsert_campos_adicionais_prata(df_campos_tratado, path_campos_prata: str):
    # Tabela não existe? cria
    if not DeltaTable.isDeltaTable(spark, path_campos_prata):
        (df_campos_tratado.write
            .format("delta")
            .mode("overwrite")
            .option("overwriteSchema", "true")
            .option("mergeSchema", "true")
            .save(path_campos_prata))
        spark.sql(f"""
          ALTER TABLE delta.`{path_campos_prata}`
          SET TBLPROPERTIES (
            'delta.enableChangeDataFeed'='true',
            'delta.autoOptimize.optimizeWrite'='true',
            'delta.autoOptimize.autoCompact'='true'
          )
        """)
        print(f"🆕 Criada PRATA de Campos Adicionais: {path_campos_prata}")
        return

    # Merge incremental
    cond = "t.idfuncionalidade = n.idfuncionalidade AND t.funcionalidade = n.funcionalidade"
    (DeltaTable.forPath(spark, path_campos_prata).alias("t")
        .merge(df_campos_tratado.alias("n"), cond)
        .whenMatchedUpdateAll(condition="n.referencia_data > t.referencia_data")
        .whenNotMatchedInsertAll()
        .execute())

    print("✅ Merge concluído em CAMPOS_ADICIONAIS_PRATA")

In [ ]:
def integrar_campos_no_main_prata(funcionalidade: str, path_main_prata: str, path_campos_prata: str):
    """Integra os campos adicionais na PRATA principal da funcionalidade."""
    spark.conf.set("spark.databricks.delta.schema.autoMerge.enabled", "true")
    id_col = f"id{funcionalidade}"

    if not DeltaTable.isDeltaTable(spark, path_main_prata):
        raise ValueError(f"Tabela principal da PRATA não existe: {path_main_prata}")

    df_check = spark.read.format("delta").load(path_main_prata)
    cols_existentes = df_check.columns

    # Garante colunas de referência no main
    if "referencia_campos_adicionais" not in cols_existentes or "referencia_data_campos_adicionais" not in cols_existentes:
        print(f"🩹 Adicionando colunas de referência em {path_main_prata}")
        (
            df_check
            .withColumn("referencia_campos_adicionais", F.lit(None).cast(T.StringType()))
            .withColumn("referencia_data_campos_adicionais", F.lit(None).cast(T.TimestampType()))
            .write
            .format("delta")
            .mode("overwrite")
            .option("mergeSchema", "true")
            .save(path_main_prata)
        )

    df_main_ids = spark.read.format("delta").load(path_main_prata).select(id_col).dropDuplicates()
    df_campos   = spark.read.format("delta").load(path_campos_prata)

    cols_func = [c for c in df_campos.columns if c.startswith(f"{funcionalidade}_")] + CHAVES_GRUPO
    df_campos_f = (
        df_campos
          .select(*[c for c in cols_func if c in df_campos.columns])
          .filter(F.col("funcionalidade") == funcionalidade)
    )

    # mantém apenas IDs existentes no main
    df_campos_f = df_campos_f.join(
        df_main_ids.select(F.col(id_col).alias("idfuncionalidade")),
        on="idfuncionalidade", how="inner"
    )

    # renomeia refs
    df_campos_f = (
        df_campos_f
          .withColumnRenamed("idfuncionalidade", "id_join_tmp")
          .withColumnRenamed("referencia",      "referencia_campos_adicionais")
          .withColumnRenamed("referencia_data", "referencia_data_campos_adicionais")
    )

    # aplica prefixo "campos_adicionais_" às colunas de conteúdo
    out_cols = [
        F.col("id_join_tmp").alias(id_col),
        F.col("funcionalidade"),
        F.col("referencia_campos_adicionais"),
        F.col("referencia_data_campos_adicionais"),
    ]
    for c in df_campos_f.columns:
        if c not in ["id_join_tmp","funcionalidade","referencia_campos_adicionais","referencia_data_campos_adicionais"]:
            out_cols.append(F.col(c).alias(f"campos_adicionais_{c}"))
    df_out = df_campos_f.select(*out_cols)

    # MERGE → atualiza quando a ref de campos for mais nova
    delta_main = DeltaTable.forPath(spark, path_main_prata)
    cond = f"t.{id_col} = n.{id_col}"
    update_cols = [c for c in df_out.columns if c != id_col]
    set_expr = {c: f"n.{c}" for c in update_cols}

    (
        delta_main.alias("t")
        .merge(df_out.alias("n"), cond)
        .whenMatchedUpdate(
            condition="""
                n.referencia_data_campos_adicionais >
                coalesce(t.referencia_data_campos_adicionais, timestamp'1900-01-01')
            """,
            set=set_expr
        )
        .whenNotMatchedInsert(values=set_expr | {id_col: f"n.{id_col}"})
        .execute()
    )

    print(f"🔗 Vinculação aplicada em {path_main_prata} ({len(update_cols)} colunas atualizáveis)")

In [ ]:
def processar_uma_funcionalidade(path_bronze: str, path_prata: str, funcionalidade: str):
    """
    Lê incremental, PIVOTA e TRATA somente a funcionalidade pedida.
    NÃO escreve automaticamente; retorna o DF tratado.
    """
    df_bronze_inc, _, _ = ler_incremental_bronze_campos(path_bronze, path_prata, funcionalidade)
    if df_bronze_inc.rdd.isEmpty():
        print("ℹ️ Nenhum novo registro na Bronze.")
        return None

    df_bronze_inc.persist(StorageLevel.MEMORY_AND_DISK)
    print(f"✅ Registros incrementais obtidos: {df_bronze_inc.count()}")

    df_pivot, _ = _pivotar_com_prefixo_debug(df_bronze_inc)
    df_func = tratar_funcionalidade(df_pivot, funcionalidade)
    return df_func

def processar_varias_funcionalidades(path_bronze: str, path_prata: str, funcionalidades: list):
    """
    Lê incremental uma vez (por funcionalidade), pivota e trata cada uma.
    Retorna dict {funcionalidade: df_tratado}
    """
    resultado = {}
    for f in funcionalidades:
        df = processar_uma_funcionalidade(path_bronze, path_prata, f)
        if df is not None:
            resultado[f] = df
    return resultado

def rodar_pipeline_campos_adicionais(funcionalidade: str):
    print(f"\n🚀 PIPELINE CAMPOS ADICIONAIS :: {funcionalidade.upper()}")
    t0 = time.perf_counter()

    # 1) processa (ler → pivotar → tratar)
    df_trat = processar_uma_funcionalidade(path_campos_bronze, path_campos_prata, funcionalidade)
    if df_trat is None:
        print("ℹ️ Nada para processar.")
        return

    # 2) (NOVO) sanitiza ANTES do upsert
    df_out = df_trat.select(
        F.col(f"id{funcionalidade}").alias("idfuncionalidade"),
        "funcionalidade", "referencia", "referencia_data",
        *[c for c in df_trat.columns if c.startswith(f"{funcionalidade}_")]
    )
    df_out = sanitize_for_sql_endpoint(df_out)  # <- aqui!

    # 3) upsert na PRATA (campos adicionais)
    upsert_campos_adicionais_prata(df_out, path_campos_prata)

    # 4) integrar no main da PRATA
    if funcionalidade not in PATHS_MAIN_PRATA:
        raise ValueError(f"PATH principal da PRATA não mapeado para '{funcionalidade}'.")
    integrar_campos_no_main_prata(
        funcionalidade=funcionalidade,
        path_main_prata=PATHS_MAIN_PRATA[funcionalidade],
        path_campos_prata=path_campos_prata
    )

    print(f"🏁 Pipeline concluído ({funcionalidade}) em {time.perf_counter() - t0:,.2f}s")

In [ ]:
rodar_pipeline_campos_adicionais("reserva")


In [ ]:
rodar_pipeline_campos_adicionais("repasse")


In [ ]:
rodar_pipeline_campos_adicionais("precadastro")

In [ ]:
rodar_pipeline_campos_adicionais("lead")

## [**Workflow**]

In [ ]:

def cast_id_columns_to_int(df):
    """
    Converte toda coluna cujo nome começa com 'id' (case-insensitive) para Integer.
    - Somente valores estritamente numéricos são convertidos.
    - Não numéricos viram NULL (mantém consistência).
    - Se já for IntegerType, mantém.
    """
    out = df
    for f in out.schema.fields:
        c = f.name
        if c.lower().startswith("id") and not isinstance(f.dataType, T.IntegerType):
            s = F.col(c).cast("string")
            out = out.withColumn(
                c,
                F.when(F.trim(s).rlike(r"^[0-9]+$"), s.cast(T.IntegerType())).otherwise(F.lit(None).cast(T.IntegerType()))
            )
    return out

In [ ]:
def tratar_workflow_incremental(nome_tabela: str, id_col: str):
    """
    🔧 Trata tabelas de workflow (leads, precadastro, reserva, repasse, etc.)
    - Ordena por referencia_data (mais recente primeiro)
    - Cria colunas de situacao_anterior e situacao_proxima
    - Incremental automático: se existir PRATA, faz merge; senão, cria
    """

    print(f"\n🚀 Iniciando tratamento incremental do workflow '{nome_tabela}'")

    # Caminhos
    path_bronze = f"{bronze_lakehouse_path}/{nome_tabela}_workflow_bronze_cvdw"
    path_prata  = f"{prata_lakehouse_path}/{nome_tabela}_workflow_prata"

    # Verifica se a Bronze existe
    if not DeltaTable.isDeltaTable(spark, path_bronze):
        print(f"⚠️ Tabela BRONZE '{nome_tabela}' não encontrada. Pulando.")
        return

    # Lê Bronze
    df_bronze = spark.read.format("delta").load(path_bronze)
    df_bronze.createOrReplaceTempView("bronze")

    # Se PRATA não existe → cria do zero
    if not DeltaTable.isDeltaTable(spark, path_prata):
        print(f"🆕 Nenhuma PRATA existente. Criando do zero para '{nome_tabela}'.")

        df_final = spark.sql(f"""
            WITH base AS (
                SELECT
                    *,
                    LAG(situacao) OVER (PARTITION BY {id_col} ORDER BY referencia_data DESC)  AS situacao_proxima,
                    LEAD(situacao) OVER (PARTITION BY {id_col} ORDER BY referencia_data DESC) AS situacao_anterior
                FROM bronze
            )
            SELECT
                referencia,
                {id_col},
                referencia_data,
                situacao,
                situacao_anterior,
                situacao_proxima,
                data_cad,
                tempo,
                ROUND(tempo / 60, 2)   AS tempo_minutos,
                ROUND(tempo / 3600, 2) AS tempo_horas,
                ROUND(tempo / 86400, 2) AS tempo_dias,
                sistema_origem,
                CURRENT_TIMESTAMP() AS data_processamento
            FROM base
        """)
            # ✅ força ids como Integer
        df_final = cast_id_columns_to_int(df_final)

        (
            df_final.write
            .format("delta")
            .mode("overwrite")
            .option("overwriteSchema", "true")
            .save(path_prata)
        )

        print(f"✅ Tabela PRATA criada com sucesso em {path_prata}")
        return

    # Caso PRATA exista → incremental
    print(f"🔁 PRATA encontrada — realizando merge incremental...")
    df_prata = spark.read.format("delta").load(path_prata)
    df_prata.createOrReplaceTempView("prata")

    ultima_ref = df_prata.selectExpr("max(referencia_data)").first()[0]
    print(f"📅 Última referencia_data processada: {ultima_ref}")

    # Lê apenas incrementais
    df_bronze_inc = spark.sql(f"""
        SELECT *
        FROM bronze
        WHERE referencia_data > TIMESTAMP('{ultima_ref}')
    """)
    df_bronze_inc.createOrReplaceTempView("bronze_inc")

    if df_bronze_inc.isEmpty():
        print(f"ℹ️ Nenhum novo registro incremental em '{nome_tabela}'.")
        return

    # Processa os incrementais com SQL
    df_tratado = spark.sql(f"""
        WITH base AS (
            SELECT
                *,
                LAG(situacao) OVER (PARTITION BY {id_col} ORDER BY referencia_data DESC)  AS situacao_proxima,
                LEAD(situacao) OVER (PARTITION BY {id_col} ORDER BY referencia_data DESC) AS situacao_anterior
            FROM bronze_inc
        )
        SELECT
            referencia,
            {id_col},
            referencia_data,
            situacao,
            situacao_anterior,
            situacao_proxima,
            data_cad,
            tempo,
            ROUND(tempo / 60, 2)   AS tempo_minutos,
            ROUND(tempo / 3600, 2) AS tempo_horas,
            ROUND(tempo / 86400, 2) AS tempo_dias,
            sistema_origem,
            CURRENT_TIMESTAMP() AS data_processamento
        FROM base
    """)
    df_tratado = cast_id_columns_to_int(df_tratado)
    # Faz o merge incremental no Delta
    delta = DeltaTable.forPath(spark, path_prata)
    (
        delta.alias("t")
        .merge(df_tratado.alias("n"), "t.referencia = n.referencia")
        .whenMatchedUpdateAll()
        .whenNotMatchedInsertAll()
        .execute()
    )

    print(f"✅ Incremento concluído com sucesso em '{nome_tabela}_workflow_prata'.")
    print("───────────────────────────────────────────────")


In [ ]:
tratar_workflow_incremental("precadastros", "idprecadastro")

In [ ]:
tratar_workflow_incremental("leads", "idlead")

In [ ]:
tratar_workflow_incremental("repasse", "idrepasse")

In [ ]:
tratar_workflow_incremental("reservas", "idreserva")

## [**Tratamento Tabelas Dimensões**]

#### [**Unidades**]

In [ ]:
# ================================================================
# ⚙️ CAMINHOS FÍSICOS DAS TABELAS BRONZE
# ================================================================
path_unidades = f"{bronze_lakehouse_path}/unidades_bronze_cvdw"
# ================================================================
# ⚙️ CAMINHOS FÍSICOS DAS TABELAS PRATA
# ================================================================
path_unidades_prata  = f"{prata_lakehouse_path}/unidades_prata"

In [ ]:
# ------------------- UNIDADES -------------------
df_unidades_bronze = ler_incremental_bronze(
    nome_tabela="unidades",
    path_bronze=path_unidades,
    path_prata=path_unidades_prata,
    id_col="idunidade"
)
if df_unidades_bronze is not None and not df_unidades_bronze.rdd.isEmpty():
    print(f"✅ df_unidades_bronze criado com sucesso — {df_unidades_bronze.count()} registros incrementais.\n")
else:
    print("⚠️ Nenhum dado incremental carregado para 'unidades'.\n")

In [ ]:
df_unidades_bronze       = normalizar_nomes_colunas(df_unidades_bronze)

In [ ]:
# exclusões usando nomes *normalizados* (a função resolve para o nome real)
excluir_valor = []
excluir_data  = ["situacao_reservada_vencimento"]

df_unidades_bronze = tratar_valores_e_datas(
    df_unidades_bronze,
    excluir_valor=excluir_valor,
    excluir_data=excluir_data,
    explicit_prefix=None,          # sem prefixo
    limitar_ao_prefixo=False,      # varre todas as colunas
    auto_prefix_listas=False,      # não tenta prefixar listas
    match_normalizado_nas_listas=True  # casa nomes normalizados com os reais
)

print("✅ RESERVA tratados e tipados com sucesso.")
df_unidades_bronze.printSchema()


In [ ]:
verificar_ids_com_virgula(df_unidades_bronze, "UNIDADES")

In [ ]:
# Ex.: UNIDADES
df_unidade_prata = preparar_para_prata(df_unidades_bronze, coluna_id="idunidade", sistema_origem="CVCRM")

In [ ]:
print("🏷️ Gerando coluna 'sigla_empreendimento' com base no nome...")

df_unidade_prata = (
    df_unidade_prata
    .withColumn(
        "sigla_empreendimento",
        F.when(F.upper(F.col("nome_empreendimento")).like("%FSA%"), "FSA")
         .when(F.upper(F.col("nome_empreendimento")).like("%AGL%"), "AGL")
         .when(F.upper(F.col("nome_empreendimento")).like("%DF%"),  "DF")
         .when(F.upper(F.col("nome_empreendimento")).like("%OCD%"), "OCD")
         .when(F.upper(F.col("nome_empreendimento")).like("%GOI%"), "GOI")
         .when(F.upper(F.col("nome_empreendimento")).like("%GOY%"), "GOY")
         .when(F.upper(F.col("nome_empreendimento")).like("%RETOM%"), "RETOMADO")
         .otherwise("OUTROS")
    )
)

print("✅ Coluna 'sigla_empreendimento' criada com sucesso a partir do nome.")

In [ ]:
print("🏷️ Gerando coluna 'nome_empreendimento_reduzido' com base no nome...")

df_unidade_prata = (
    df_unidade_prata
    .withColumn(
        "nome_empreendimento_reduzido",
        F.trim(F.split(F.col("nome_empreendimento"), "-").getItem(0))
    )
)

print("✅ Coluna 'sigla_empreendimento' criada com sucesso a partir do nome.")

In [ ]:
upsert_prata(df_unidade_prata, f"{prata_lakehouse_path}/unidade_prata", coluna_id="idunidade", zorder_cols=["idunidade","referencia_data"])

#### [**Vendas**]

In [ ]:
# ================================================================
# ⚙️ CAMINHOS FÍSICOS DAS TABELAS BRONZE
# ================================================================
path_vendas = f"{bronze_lakehouse_path}/vendas_bronze_cvdw"
# ================================================================
# ⚙️ CAMINHOS FÍSICOS DAS TABELAS PRATA
# ================================================================
path_vendas_prata  = f"{prata_lakehouse_path}/vendas_prata"


In [ ]:
# ------------------- VENDAS -------------------
df_vendas_bronze = ler_incremental_bronze(
    nome_tabela="vendas_bronze_cvdw",
    path_bronze=path_vendas,
    path_prata=path_vendas_prata,
    id_col="referencia"
)
if df_vendas_bronze is not None and not df_vendas_bronze.rdd.isEmpty():
    print(f"✅ df_vendas_bronze criado com sucesso — {df_vendas_bronze.count()} registros incrementais.\n")
else:
    print("⚠️ Nenhum dado incremental carregado para 'vendas'.\n")

In [ ]:
df_vendas_bronze       = normalizar_nomes_colunas(df_vendas_bronze)

In [ ]:
# exclusões usando nomes *normalizados* (a função resolve para o nome real)
excluir_valor = []
excluir_data  = []

df_vendas_bronze = tratar_valores_e_datas(
    df_vendas_bronze,
    excluir_valor=excluir_valor,
    excluir_data=excluir_data,
    explicit_prefix=None,          # sem prefixo
    limitar_ao_prefixo=False,      # varre todas as colunas
    auto_prefix_listas=False,      # não tenta prefixar listas
    match_normalizado_nas_listas=True  # casa nomes normalizados com os reais
)

print("✅ Vendas tratados e tipados com sucesso.")
df_vendas_bronze.printSchema()


In [ ]:
verificar_ids_com_virgula(df_vendas_bronze, "VENDAS")

In [ ]:
# Ex.: VENDAS
df_vendas_prata = preparar_para_prata(df_vendas_bronze, coluna_id="referencia", sistema_origem="CVCRM")
df_vendas_prata.printSchema()

In [ ]:
upsert_prata(df_vendas_prata, f"{prata_lakehouse_path}/venda_prata", coluna_id="referencia", zorder_cols=["referencia","referencia_data"])

#### [**Reserva Registro Flags**]

In [ ]:
# ================================================================
# ⚙️ CAMINHOS FÍSICOS DAS TABELAS BRONZE
# ================================================================
path_reservas_registros_flags = f"{bronze_lakehouse_path}/reservas_registros_flags_bronze_cvdw"

# ================================================================
# ⚙️ CAMINHOS FÍSICOS DAS TABELAS PRATA
# ================================================================
path_reservas_registros_flags_prata  = f"{prata_lakehouse_path}/reservas_registros_flags_prata"


In [ ]:
# ------------------- Reservas Registros Flags -------------------
df_reservas_registros_flags_bronze = ler_incremental_bronze(
    nome_tabela="reservas_registros_flags_bronze_cvdw",
    path_bronze=path_reservas_registros_flags,
    path_prata=path_reservas_registros_flags_prata,
    id_col="referencia"
)
if df_reservas_registros_flags_bronze is not None and not df_reservas_registros_flags_bronze.rdd.isEmpty():
    print(f"✅ df_vendas_bronze criado com sucesso — {df_reservas_registros_flags_bronze.count()} registros incrementais.\n")
else:
    print("⚠️ Nenhum dado incremental carregado para 'vendas'.\n")

In [ ]:
df_reservas_registros_flags_bronze       = normalizar_nomes_colunas(df_reservas_registros_flags_bronze)

In [ ]:
# exclusões usando nomes *normalizados* (a função resolve para o nome real)
excluir_valor = []
excluir_data  = []

df_reservas_registros_flags_bronze = tratar_valores_e_datas(
    df_reservas_registros_flags_bronze,
    excluir_valor=excluir_valor,
    excluir_data=excluir_data,
    explicit_prefix=None,          # sem prefixo
    limitar_ao_prefixo=False,      # varre todas as colunas
    auto_prefix_listas=False,      # não tenta prefixar listas
    match_normalizado_nas_listas=True  # casa nomes normalizados com os reais
)

print("✅ df_reservas_registros_flags_bronze tratados e tipados com sucesso.")
df_reservas_registros_flags_bronze.printSchema()


In [ ]:
display(df_reservas_registros_flags_bronze)

In [ ]:
upsert_prata(df_reservas_registros_flags_bronze, f"{prata_lakehouse_path}/reservas_registros_flags_prata", coluna_id="idreserva", zorder_cols=["idreserva"])

#### [**Corretores**]

In [ ]:
# ================================================================
# ⚙️ CAMINHOS FÍSICOS DAS TABELAS BRONZE
# ================================================================
path_corretores = f"{bronze_lakehouse_path}/corretores_bronze_cvdw"

# ================================================================
# ⚙️ CAMINHOS FÍSICOS DAS TABELAS PRATA
# ================================================================
path_corretores_prata  = f"{prata_lakehouse_path}/corretores_prata"


In [ ]:
# ------------------- Corretores -------------------
df_corretores_bronze = ler_incremental_bronze(
    nome_tabela="corretores",
    path_bronze=path_corretores,
    path_prata=path_corretores_prata,
    id_col="referencia"
)
if df_corretores_bronze is not None and not df_corretores_bronze.rdd.isEmpty():
    print(f"✅ df_corretores_bronze criado com sucesso — {df_corretores_bronze.count()} registros incrementais.\n")
else:
    print("⚠️ Nenhum dado incremental carregado para 'vendas'.\n")

In [ ]:
df_corretores_bronze       = normalizar_nomes_colunas(df_corretores_bronze)

In [ ]:
# exclusões usando nomes *normalizados* (a função resolve para o nome real)
excluir_valor = []
excluir_data  = []

df_corretores_bronze = tratar_valores_e_datas(
    df_corretores_bronze,
    excluir_valor=excluir_valor,
    excluir_data=excluir_data,
    explicit_prefix=None,          # sem prefixo
    limitar_ao_prefixo=False,      # varre todas as colunas
    auto_prefix_listas=False,      # não tenta prefixar listas
    match_normalizado_nas_listas=True  # casa nomes normalizados com os reais
)

print("✅ df_corretores_bronze tratados e tipados com sucesso.")
df_corretores_bronze.printSchema()

In [ ]:
upsert_prata(df_corretores_bronze, f"{prata_lakehouse_path}/corretores_prata", coluna_id="referencia", zorder_cols=["referencia","referencia_data"])

#### **Imobiliarias**

In [ ]:
# ================================================================
# ⚙️ CAMINHOS FÍSICOS DAS TABELAS BRONZE
# ================================================================
path_imobiliarias = f"{bronze_lakehouse_path}/imobiliarias_bronze_cvdw"

# ================================================================
# ⚙️ CAMINHOS FÍSICOS DAS TABELAS PRATA
# ================================================================
path_imobiliarias_prata  = f"{prata_lakehouse_path}/imobiliarias_prata"

In [ ]:
# ------------------- Corretores -------------------
df_imobiliarias_bronze = ler_incremental_bronze(
    nome_tabela="imobiliarias",
    path_bronze=path_imobiliarias,
    path_prata=path_imobiliarias_prata,
    id_col="referencia"
)
if df_imobiliarias_bronze is not None and not df_imobiliarias_bronze.rdd.isEmpty():
    print(f"✅ df_imobiliarias_bronze criado com sucesso — {df_imobiliarias_bronze.count()} registros incrementais.\n")
else:
    print("⚠️ Nenhum dado incremental carregado para 'Imobiliaria'.\n")

In [ ]:
df_imobiliarias_bronze       = normalizar_nomes_colunas(df_imobiliarias_bronze)

In [ ]:
# exclusões usando nomes *normalizados* (a função resolve para o nome real)
excluir_valor = []
excluir_data  = []

df_imobiliarias_bronze = tratar_valores_e_datas(
    df_imobiliarias_bronze,
    excluir_valor=excluir_valor,
    excluir_data=excluir_data,
    explicit_prefix=None,          # sem prefixo
    limitar_ao_prefixo=False,      # varre todas as colunas
    auto_prefix_listas=False,      # não tenta prefixar listas
    match_normalizado_nas_listas=True  # casa nomes normalizados com os reais
)

print("✅ df_corretores_bronze tratados e tipados com sucesso.")
df_imobiliarias_bronze.printSchema()

In [ ]:
upsert_prata(df_imobiliarias_bronze, f"{prata_lakehouse_path}/imobiliarias_prata", coluna_id="referencia", zorder_cols=["referencia","referencia_data"])

#### [**Distrato**]

In [ ]:
# ================================================================
# ⚙️ CAMINHOS FÍSICOS DAS TABELAS BRONZE
# ================================================================
path_distratos = f"{bronze_lakehouse_path}/distratos_cvdw"

# ================================================================
# ⚙️ CAMINHOS FÍSICOS DAS TABELAS PRATA
# ================================================================
path_distratos_prata  = f"{prata_lakehouse_path}/distratos_prata"

In [ ]:
# ------------------- Corretores -------------------
df_distratos_bronze = ler_incremental_bronze(
    nome_tabela="distratos",
    path_bronze=path_distratos,
    path_prata=path_distratos_prata,
    id_col="referencia"
)
if df_distratos_bronze is not None and not df_distratos_bronze.rdd.isEmpty():
    print(f"✅ df_distratos_bronze criado com sucesso — {df_distratos_bronze.count()} registros incrementais.\n")
else:
    print("⚠️ Nenhum dado incremental carregado para 'Imobiliaria'.\n")

In [ ]:
df_distratos_bronze       = normalizar_nomes_colunas(df_distratos_bronze)

In [ ]:
# exclusões usando nomes *normalizados* (a função resolve para o nome real)
excluir_valor = []
excluir_data  = []

df_distratos_bronze = tratar_valores_e_datas(
    df_distratos_bronze,
    excluir_valor=excluir_valor,
    excluir_data=excluir_data,
    explicit_prefix=None,          # sem prefixo
    limitar_ao_prefixo=False,      # varre todas as colunas
    auto_prefix_listas=False,      # não tenta prefixar listas
    match_normalizado_nas_listas=True  # casa nomes normalizados com os reais
)

print("✅ df_corretores_bronze tratados e tipados com sucesso.")
df_distratos_bronze.printSchema()

In [ ]:
upsert_prata(df_distratos_bronze, f"{prata_lakehouse_path}/distratos_prata", coluna_id="referencia", zorder_cols=["referencia","referencia_data"])